# BEGINNER LEVEL TASK - 3
# Music Recommendation

# Importing the Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Loading the dataset

In [9]:
members = pd.read_csv(".\members.csv")
submission = pd.read_csv(".\sample_submission.csv",nrows=20000)
songs_info = pd.read_csv(".\song_extra_info.csv")
songs = pd.read_csv(".\songs.csv",nrows=20000)
train = pd.read_csv( "./train.csv",nrows=20000)
test = pd.read_csv("./test.csv")

# **Exploratory Data Analysis**

In [10]:
members.sample(5)

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
24054,AyLluAjcv++USVU/cgONguzYFfFeSosTW9c35KjJTPU=,1,0,NaN,7,20160217,20170905
26985,dJmsq5q01qLJN2IqWpEO882Qv7AFgJcqSE/s0DOU6y4=,1,17,female,3,20160624,20170130
8253,2aq27fhziE3vQBA9wusbb6qbolVR9MCECauVeLOs4qs=,8,18,male,4,20161229,20170129
7745,4oXa8IqoS8ziQFMa2ftp7+4M9Y437MiBG3hBoWk9oVE=,4,20,male,3,20160106,20170617
4229,mDqF17eowvmEMYsk0Qx+vteDEDwsDyUTupDLJLazqjY=,3,24,male,9,20170121,20170426


In [11]:
submission.sample(5)

,id,target
8162,8162,0.5
17887,17887,0.5
5464,5464,0.5
13902,13902,0.5
15206,15206,0.5


In [12]:
songs_info.sample(5)

,song_id,name,isrc
1530556,8Raw2UrYSK2yoOGzQTQqXDMXyQWF/UqnX9wohkxZydE=,Send Me The Pillow You Dream On,DEPZ69490406
1030864,bpbJEFsUsol0IIa9M8MYvm2i1SBpHeGnMPtgqTnJDBg=,微醺,NaN
869636,FNwvpx261BVp6gftRJr00xcNvn7UDhqzKEQaFASTNnc=,瘋狂假面,TWB830500107
1191100,ySkmaUz0o4BmYdT3cPYA3NJVv3mNA954zFyqK0p+7tk=,Two Friends,USA561111934
1477678,Ho1WPQSgjn3UEovtxn6DtH/1GF0v6eDEDvHcd2xoNYI=,비상 (Emergency) [In the Style of Koyote],KRB361210770


In [13]:
songs.sample(5)

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
6481,2DpgBtp1vWy4lbZdf42qP/G8qxECByaDDUZxQ/My2zQ=,179350,465,Kim Bo Kyung,NaN,NaN,31.0
15405,235zbFHu/JYYE2P+wWIXDN73k1lFWgNRx9L4ki1RGqw=,277002,921,Takagi Masakatsu (高木正勝),Masakatsu Takagi,NaN,17.0
13615,uMmro5VULe0oNUQAVWflLXo/GwC0LK6G28/97x+T9yM=,249986,465,姜育恆 (Keung Hang),李壽全/武雄,李壽全/武雄,3.0
1160,3+tKkL0xiKeZTyzvbf6gJkf5gi+aYyQIPnlfKJ/r77Q=,274877,465,華夏流行歌曲精選集,NaN,NaN,3.0
7896,ZuRGpHR3oY5fnTijrXSzv+F2kQOqyloyHDxoe0gqjiM=,254815,1152,Matthew Lien,NaN,NaN,-1.0


In [14]:
train.sample(5)

,msno,song_id,source_system_tab,source_screen_name,source_type,target
7730,7H8Teyq/bk0vHVwbjT2+Otaz8vHTgdJHmfxwYm2P/K8=,2wM8GWCiMKvbzjhNfNEI3tA5KWsGOUK4cTnuEVYDcUQ=,my library,Local playlist more,local-playlist,1
11381,oyNW4PvdInWCHV8royT4Iwe/eswMHCrXCcq69eE1Zf8=,lLAqtsvNStONkkrs0X+ASQrz/nH8GOZ9UH6MQjA70QU=,my library,Local playlist more,local-library,0
14524,HARygVqGgCsyWij/ltgSgtUXEqVcNggYXiGAoaWxnFA=,9t9+DPdDvWrge1ZOUlcJvLmOfnpa4NtNEVY0rBc/91I=,radio,Radio,radio,1
12850,cEhCRr+hGkQvqDPRd8eK47vJ9d7Kns+sMu0fs+oIRuQ=,YSIgT0sONs9cJGJcpkFPSDdZtUwabSjwgkvZoyfeEIo=,my library,Local playlist more,local-playlist,1
16084,b4ioSr7PCjSNL1HNyzfSEoW9WT6PLGIrpUcg8wGdE9Q=,fCCmIa0Y5m+MCGbQga31MOLTIqi7ddgXvkjFPmfslGw=,discover,Online playlist more,online-playlist,0


In [15]:
test.sample(5)

,id,msno,song_id,source_system_tab,source_screen_name,source_type
620753,620753,x9JYlJSBNsO+ikp41JA/A3QgISfzekspIBUutZUwCrA=,RtruXqI7qVe60bQrQ+VE3fgdcRnL+If4fnd29LlR3J4=,my library,Local playlist more,local-library
24312,24312,omj9hjUFiWmCCpiTn0BWFFSofE+ZdkZkx4jlgWgo1Wo=,sYTikvUS4j1SLw2v3WggJhP1U4rcVl803qOktFgLv9o=,discover,Online playlist more,online-playlist
1695760,1695760,Wc3sX9RyaEozCuXyy6daSVYgXAseY6HccmvMwRAmkuk=,HMJU+Q92zjWOHDg3KO+/WHWdFf8UP2f3hpJBKeGMbl0=,discover,Album more,album
633748,633748,LefrUriGp2YsuXM07RE20d4mSHJB75/oSi0AO96OqEg=,z1bWPlWOviDqx9TKHze5LV6Hf40yNY2PhUH5bqV4JMc=,discover,Discover Chart,online-playlist
1786344,1786344,VmVNLJQRDxLsfXpE9ViQWm/H7P/gB/sjye6tdBBwf9s=,kMIJnH+Xma633KX+coW+4Jy517FE7ayQfvEWue2knfQ=,discover,Discover Chart,online-playlist


## Checking the null values

In [16]:
 members.isnull().sum()

msno                          0
city                          0
bd                            0
gender                    19902
registered_via                0
registration_init_time        0
expiration_date               0
dtype: int64

In [17]:
submission.isnull().sum()

id        0
target    0
dtype: int64

In [18]:
songs_info.isnull().sum()

song_id         0
name            2
isrc       136548
dtype: int64

In [19]:
songs.isnull().sum()

song_id            0
song_length        0
genre_ids        346
artist_name        0
composer        8382
lyricist       14332
language           0
dtype: int64

In [20]:
train.isnull().sum()

msno                    0
song_id                 0
source_system_tab      67
source_screen_name    576
source_type            50
target                  0
dtype: int64

In [21]:
test.isnull().sum()

id                         0
msno                       0
song_id                    0
source_system_tab       8442
source_screen_name    162883
source_type             7297
dtype: int64

In [22]:
songs.shape

(20000, 7)

In [23]:
songs.isnull().mean()

song_id        0.0000
song_length    0.0000
genre_ids      0.0173
artist_name    0.0000
composer       0.4191
lyricist       0.7166
language       0.0000
dtype: float64

## Replacing the nan values with empty string for genre_ids, composer, and lyricist 

In [24]:
songs['genre_ids'].fillna(' ',inplace=True)
songs['composer'].fillna(' ',inplace=True)
songs['lyricist'].fillna(' ',inplace=True)

In [25]:
songs.isnull().sum()

song_id        0
song_length    0
genre_ids      0
artist_name    0
composer       0
lyricist       0
language       0
dtype: int64

# Droping some non-required columns from train data

In [26]:
train = train.drop(['source_system_tab','source_screen_name','source_type'],axis=1)
train.head()

,msno,song_id,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,1


# Renaming the column name

In [27]:
train.rename(columns={'msno':'user_id'},inplace=True)
train.head()

,user_id,song_id,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,1


In [28]:
train.shape

(20000, 3)

# **Basic Preprocessing**

# Merging songs and train dataset

In [29]:
df = train.merge(songs, on='song_id')

# Dropping 'song_length','language' column

In [30]:
df=df.drop(['song_length','language'],axis=1)
df.head()

,user_id,song_id,target,genre_ids,artist_name,composer,lyricist
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3Hg5kugV1S0wzEVLAEfqjIV5UHzb7bCrdBRQlGygLvU=,1,1259,Desiigner,Sidney Selby| Adnan Khan,
1,hZyOA+0yqClPLt6uIEndf8fG8szH/95eKMbaxLE5z30=,3Hg5kugV1S0wzEVLAEfqjIV5UHzb7bCrdBRQlGygLvU=,1,1259,Desiigner,Sidney Selby| Adnan Khan,
2,0LhkakIQDn36HZXI6ClQSO7W7jkpZAy+9MvYgPOZGrA=,skehue/d/R59G71dXYpntDwdjRRPlweN3JE8g40TgZU=,1,458,莊心妍,鄭建浩,鄭建浩
3,MofmAMt7P8LIcF4+LLlcjyIhYUzmv13L/LRwYFxiGYE=,skehue/d/R59G71dXYpntDwdjRRPlweN3JE8g40TgZU=,1,458,莊心妍,鄭建浩,鄭建浩
4,U9Z+N+szYGJHTPMn/C0V7yIyIC24fDI0RDRWChXATkg=,skehue/d/R59G71dXYpntDwdjRRPlweN3JE8g40TgZU=,0,458,莊心妍,鄭建浩,鄭建浩


# Merging the songs_info column

In [32]:
df = df.merge(songs_info, on='song_id').drop('isrc', axis=1)
df.head()

,user_id,song_id,target,genre_ids,artist_name,composer,lyricist,name
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3Hg5kugV1S0wzEVLAEfqjIV5UHzb7bCrdBRQlGygLvU=,1,1259,Desiigner,Sidney Selby| Adnan Khan,,Panda
1,hZyOA+0yqClPLt6uIEndf8fG8szH/95eKMbaxLE5z30=,3Hg5kugV1S0wzEVLAEfqjIV5UHzb7bCrdBRQlGygLvU=,1,1259,Desiigner,Sidney Selby| Adnan Khan,,Panda
2,0LhkakIQDn36HZXI6ClQSO7W7jkpZAy+9MvYgPOZGrA=,skehue/d/R59G71dXYpntDwdjRRPlweN3JE8g40TgZU=,1,458,莊心妍,鄭建浩,鄭建浩,我過的很好
3,MofmAMt7P8LIcF4+LLlcjyIhYUzmv13L/LRwYFxiGYE=,skehue/d/R59G71dXYpntDwdjRRPlweN3JE8g40TgZU=,1,458,莊心妍,鄭建浩,鄭建浩,我過的很好
4,U9Z+N+szYGJHTPMn/C0V7yIyIC24fDI0RDRWChXATkg=,skehue/d/R59G71dXYpntDwdjRRPlweN3JE8g40TgZU=,0,458,莊心妍,鄭建浩,鄭建浩,我過的很好


# Renaming the columns

In [33]:
df.rename(columns={'name':'song_name'}, inplace=True)
df.head()

,user_id,song_id,target,genre_ids,artist_name,composer,lyricist,song_name
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3Hg5kugV1S0wzEVLAEfqjIV5UHzb7bCrdBRQlGygLvU=,1,1259,Desiigner,Sidney Selby| Adnan Khan,,Panda
1,hZyOA+0yqClPLt6uIEndf8fG8szH/95eKMbaxLE5z30=,3Hg5kugV1S0wzEVLAEfqjIV5UHzb7bCrdBRQlGygLvU=,1,1259,Desiigner,Sidney Selby| Adnan Khan,,Panda
2,0LhkakIQDn36HZXI6ClQSO7W7jkpZAy+9MvYgPOZGrA=,skehue/d/R59G71dXYpntDwdjRRPlweN3JE8g40TgZU=,1,458,莊心妍,鄭建浩,鄭建浩,我過的很好
3,MofmAMt7P8LIcF4+LLlcjyIhYUzmv13L/LRwYFxiGYE=,skehue/d/R59G71dXYpntDwdjRRPlweN3JE8g40TgZU=,1,458,莊心妍,鄭建浩,鄭建浩,我過的很好
4,U9Z+N+szYGJHTPMn/C0V7yIyIC24fDI0RDRWChXATkg=,skehue/d/R59G71dXYpntDwdjRRPlweN3JE8g40TgZU=,0,458,莊心妍,鄭建浩,鄭建浩,我過的很好


# Number of vales in gener_ids

In [34]:
df['genre_ids'].value_counts()

465                    710
458                    400
444                     65
1609                    57
921                     49
359                     32
                        26
139                     24
2022                    21
1259                    20
2122                    13
139|125|109              7
726                      7
451                      7
437                      7
958                      6
786|947                  6
465|1259                 4
1011                     4
786                      4
947                      4
691                      3
921|465                  3
430                      3
921|458                  2
458|1287                 2
698                      2
444|1259                 2
829                      2
850                      2
1152                     1
880|458                  1
465|829                  1
864|857|850|843          1
465|798                  1
474                      1
864|850|726|857|843      1
3

# Replacing this | symbol into empty string

In [35]:
df['genre_ids']=df['genre_ids'].str.replace('|',' ',regex=True)
df['genre_ids'].value_counts()

465                    710
458                    400
444                     65
1609                    57
921                     49
359                     32
                        26
139                     24
2022                    21
1259                    20
2122                    13
139 125 109              7
726                      7
451                      7
437                      7
958                      6
786 947                  6
465 1259                 4
1011                     4
786                      4
947                      4
691                      3
921 465                  3
430                      3
921 458                  2
458 1287                 2
698                      2
444 1259                 2
829                      2
850                      2
1152                     1
880 458                  1
465 829                  1
864 857 850 843          1
465 798                  1
474                      1
864 850 726 857 843      1
3

# Replacing this | symbol from every column

In [36]:
df['artist_name'] = df['artist_name'].str.replace('|', ' ', regex=True)
df['artist_name'] = df['artist_name'].str.replace('/', ' ', regex=True)
df['composer'] = df['composer'].str.replace('|', ' ', regex=True)
df['composer'] = df['composer'].str.replace('/', ' ', regex=True)
df['lyricist'] = df['lyricist'].str.replace('|', ' ', regex=True)
df['lyricist'] = df['lyricist'].str.replace('/', ' ', regex=True)

# Text Preprocessing

In [37]:
df['artist_name'] = df['artist_name'].str.lower()
df['composer'] = df['composer'].str.lower()
df['lyricist'] = df['lyricist'].str.lower()
df['songs_details'] = df['artist_name'] + '' + df['composer'] + df['lyricist']
df.head()

,user_id,song_id,target,genre_ids,artist_name,composer,lyricist,song_name,songs_details
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3Hg5kugV1S0wzEVLAEfqjIV5UHzb7bCrdBRQlGygLvU=,1,1259,desiigner,sidney selby adnan khan,,Panda,desiignersidney selby adnan khan
1,hZyOA+0yqClPLt6uIEndf8fG8szH/95eKMbaxLE5z30=,3Hg5kugV1S0wzEVLAEfqjIV5UHzb7bCrdBRQlGygLvU=,1,1259,desiigner,sidney selby adnan khan,,Panda,desiignersidney selby adnan khan
2,0LhkakIQDn36HZXI6ClQSO7W7jkpZAy+9MvYgPOZGrA=,skehue/d/R59G71dXYpntDwdjRRPlweN3JE8g40TgZU=,1,458,莊心妍,鄭建浩,鄭建浩,我過的很好,莊心妍鄭建浩鄭建浩
3,MofmAMt7P8LIcF4+LLlcjyIhYUzmv13L/LRwYFxiGYE=,skehue/d/R59G71dXYpntDwdjRRPlweN3JE8g40TgZU=,1,458,莊心妍,鄭建浩,鄭建浩,我過的很好,莊心妍鄭建浩鄭建浩
4,U9Z+N+szYGJHTPMn/C0V7yIyIC24fDI0RDRWChXATkg=,skehue/d/R59G71dXYpntDwdjRRPlweN3JE8g40TgZU=,0,458,莊心妍,鄭建浩,鄭建浩,我過的很好,莊心妍鄭建浩鄭建浩


In [38]:
df.user_id.value_counts()

EozJegFxTFIWDb9aJ7O8kSUHAgx4ZIvqf7IuN5Zck50=    19
V5U4EGk2kaSKaUGSwhU6g3HBefxflEvAy1vWPu6UBQs=    18
Bwg9yS76qujJJeKsYSzfJrMlkjK5Ui7KFkgUcjuXRCg=    12
W9NYSCff57nmfyYCiX6IbW0/G3YuwC18h/rld+BGxMY=    11
UzlQoa9tdrcpYdh4wksoh+SpWCFcKvRGPA+xLNqghmo=    11
                                                ..
rb7TT328utsdnd8COyhstig0zciXIURo7M464E60EHg=     1
hSn7jMfIURFu+1W3PDIDTxbhM5SxRg9VFRoH23Rm2Ic=     1
yrMfQXudhDaA/bOePZtkKErbjZc5pALG79FHPayEy5U=     1
iP3eF1In0rH61CfgVmWVYj4CgFcQQ0iVZG7MBA+Plgo=     1
j2Sx5B7BrjqCiT3ZwWK4AvepwM14QEalhTPi2/sgdG4=     1
Name: user_id, Length: 975, dtype: int64

# Checking for duplicate values

In [39]:
 df.duplicated().sum()

0

# Creating a copy file before performing similarity

In [40]:
temp_df = df.copy()
temp_df.head()

,user_id,song_id,target,genre_ids,artist_name,composer,lyricist,song_name,songs_details
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3Hg5kugV1S0wzEVLAEfqjIV5UHzb7bCrdBRQlGygLvU=,1,1259,desiigner,sidney selby adnan khan,,Panda,desiignersidney selby adnan khan
1,hZyOA+0yqClPLt6uIEndf8fG8szH/95eKMbaxLE5z30=,3Hg5kugV1S0wzEVLAEfqjIV5UHzb7bCrdBRQlGygLvU=,1,1259,desiigner,sidney selby adnan khan,,Panda,desiignersidney selby adnan khan
2,0LhkakIQDn36HZXI6ClQSO7W7jkpZAy+9MvYgPOZGrA=,skehue/d/R59G71dXYpntDwdjRRPlweN3JE8g40TgZU=,1,458,莊心妍,鄭建浩,鄭建浩,我過的很好,莊心妍鄭建浩鄭建浩
3,MofmAMt7P8LIcF4+LLlcjyIhYUzmv13L/LRwYFxiGYE=,skehue/d/R59G71dXYpntDwdjRRPlweN3JE8g40TgZU=,1,458,莊心妍,鄭建浩,鄭建浩,我過的很好,莊心妍鄭建浩鄭建浩
4,U9Z+N+szYGJHTPMn/C0V7yIyIC24fDI0RDRWChXATkg=,skehue/d/R59G71dXYpntDwdjRRPlweN3JE8g40TgZU=,0,458,莊心妍,鄭建浩,鄭建浩,我過的很好,莊心妍鄭建浩鄭建浩


In [41]:
temp_df.songs_details.duplicated().sum()

889

In [42]:
temp_df.shape

(1509, 9)

In [43]:
temp_df.duplicated().sum()

0

# Using TFIDF for vectorization

In [51]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')
tfidf_matrix = tfidf.fit_transform(temp_df['songs_details'])

In [52]:
tfidf_matrix

<1509x1931 sparse matrix of type '<class 'numpy.float64'>'
	with 6876 stored elements in Compressed Sparse Row format>

In [53]:
temp_df.shape

(1509, 8)

In [54]:
temp_df.reset_index(drop=True, inplace=True)

# Now for the Recommendation we will use cosine similarity

In [56]:
cosine_similarity = cosine_similarity(tfidf_matrix)

In [57]:
cosine_similarity

array([[1., 1., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

# User based Recommendation 

In [59]:
def recommend_song(song):
    song_index = temp_df[temp_df['song_name'] == song].index[0]
    distances = cosine_similarity[song_index]
    song_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:10]
    for i in song_list:
        print(temp_df.iloc[i[0]].song_name)

In [60]:
recommend_song('Fine Day')

太極拳Cypher (Tai Chi Fist Cypher)
太極拳Cypher (Tai Chi Fist Cypher)
布魯赫：第一號小提琴協奏曲/第二樂章：慢板 (Violin Concerto No. 1 in G Minor| Op. 26 - Adagio)
靠近一點點
靠近一點點
靠近一點點
靠近一點點
mozart sonata in d major k448 (2 pianos) - 2nd movement mozart sonata in d major k448 (2 pianos) - 2nd movement
We Don't Talk Any More
